## How far virtual employees live from the office?


I was doing audit of our virtual workforce in Genesys to answer the question whether our virtual employees actually live that far from the office or if it was more of the personal preference to work from home? The idea was to use Bing Maps to calculate the average time driving between office zip code and employee's home zip code.

The further analysis itself was carried out in Tableau, where I joined employee IDs back to the full details. 

In [1]:
## let's import modules to communicate with Bing Maps API and time for delay in between the requests sent. 

import urllib.request
import time
import json

bingMapsKey = "YOUR-API-CALL"
o = 'json'

To use the functionality you need to obtain API key from https://www.bingmapsportal.com/. 

In this example I use json to get output (o = 'json') but you can modify it to xml, if you have to.

In [2]:
def getCommute(homeZip):
    # adding delay  
    time.sleep(1)
    wp0 = homeZip
    
    # this is our Daly City office zip code. Add functionality to ask use to enter zip-code or change to your zip code.
    
    wp1 = '94014'
    encodedDest = urllib.parse.quote(wp1, safe='')
    routeUrl = "http://dev.virtualearth.net/REST/V1/Routes/Driving?o=" + o + "&wp.0=" + wp0 + "&wp.1=" + encodedDest + "&key=" + bingMapsKey
    request = urllib.request.Request(routeUrl)
    response = urllib.request.urlopen(request)
    r = response.read().decode(encoding="utf-8")
    result = json.loads(r)
    
    # I am using value from Travel Duration but you can replace to whatever you need.
    
    itineraryItems = result['resourceSets'][0]['resources'][0]['travelDuration']
    return itineraryItems

In [3]:
# Let's add pandas because it is the easiest way for myself to lambda in the column with output, then read csv with your data. 
# The structure of data is simple - two columns "Employee Number" and "Postal Code"

import pandas as pd
df = pd.read_csv("gtsvirtual.csv")

In [4]:
# Let's run our function on some records and add the commute time to the dataframe 

df['Commute'] = df[:5].apply(lambda row: getCommute(str(row['Postal Code'])), axis=1)

In [5]:
# Checking on the output
df[:5]

,Employee Number,Postal Code,Commute
0,LK0078017,90034,19895.0
1,1001582,90038,19816.0
2,LK0079373,90038,19816.0
3,200652,90065,19942.0
4,LK0080296,90069,20236.0


Since it works make sure to expand the slice and run the results for the entire data set.

In [6]:
#Save output to csv

df.to_csv("Output_forVFW.csv", index=False)